# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [5]:
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/tencent/Hunyuan-A13B-Instruct',
 '/google/gemma-3n-E4B-it',
 '/THUDM/GLM-4.1V-9B-Thinking',
 '/kyutai/tts-1.6b-en_fr',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/AIDC-AI/Ovis-U1-3B',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/facebook/seamless-interaction',
 '/datasets/FreedomIntelligence/ShareGPT-4o-Image',
 '/datasets/black-forest-labs/kontext-bench',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Wri

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://huggingface.co/discuss'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
black-forest-labs/FLUX.1-Kontext-dev
Updated
8 days ago
•
155k
•
1.32k
tencent/Hunyuan-A13B-Instruct
Updated
3 days ago
•
13.7k
•
709
google/gemma-3n-E4B-it
Updated
2 days ago
•
197k
•
445
THUDM/GLM-4.1V-9B-Thinking
Updated
2 days ago


In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n8 days ago\n•\n155k\n•\n1.32k\ntencent/Hunyuan-A13B-Instruct\nUpdated\n3 days ago\n•\n13.7k\n•\n709\ngoogle/gemma-3n-E4B-it\nUpdated\n2 days ago\n•\n197k\n•\n445\nTHUDM/GLM-4.1V-9B-Thinking\nUpdated\n2 days ago\n•\n7.03k\n•\n204\nkyutai/tts-1.6b-en_fr\nUpdated\n2 days ago\n•\n7.19k\n•\n164\nBrowse 1M+ models\nSpaces\nRunning\n9.25k\n9.25k\nDee

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://huggingface.co/contact'}, {'type': 'community page', 'url': 'https://huggingface.co/discuss'}, {'type': 'social media page', 'url': 'https://twitter.com/huggingface'}, {'type': 'social media page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# Welcome to Hugging Face

**The AI community building the future.**

At Hugging Face, we are dedicated to creating a vibrant platform where the machine learning community can collaborate, innovate, and excel. With resources that empower users to create, discover, and share machine learning models, datasets, and applications, Hugging Face is at the forefront of AI technology.

---

## Our Offerings

- **Models**: Browse and contribute to over **1 million+ models**.
- **Datasets**: Access and share **250,000+ datasets** for all your machine learning needs.
- **Spaces**: Deploy applications and tools seamlessly, hosting over **400,000 applications**.

### Key Features
- **Open Source**: We build our tools with the community, fostering collaborative growth in ML.
- **Enterprise Solutions**: Tailored paid services start at $20/user/month to help organizations scale their AI offerings with security and support.
- **Compute Resources**: Accelerate your ML projects with our optimized inference endpoints, starting at $0.60/hour for GPU usage.

---

## Community and Collaboration

Hugging Face is not just a platform; it's a thriving community of over **50,000 organizations** globally, including tech giants like **Google**, **Microsoft**, and **Amazon**, all utilizing our models and datasets for cutting-edge AI development. 

We prioritize collaboration and encourage users to build **ML portfolios**, share their work, and contribute to the open-source ecosystem.

---

## Company Culture

At Hugging Face, we believe in the power of collaboration, innovation, and diversity. Our culture is rooted in openness and inclusivity, creating an environment where all voices are heard and valued. We focus on continuous learning, pushing the boundaries of AI technologies, and supporting our community in their growth journeys.

---

## Careers at Hugging Face

Join a passionate and dynamic team that is shaping the future of AI! We are always looking for talented individuals dedicated to innovation and collaboration. Explore current openings on our [careers page](#) and be a part of our mission to democratize AI.

---

### Connect With Us

- **Website**: [Hugging Face](https://huggingface.co)
- **GitHub**: [Hugging Face on GitHub](https://github.com/huggingface)
- **Twitter**: [@HuggingFace](https://twitter.com/huggingface)
- **LinkedIn**: [Hugging Face on LinkedIn](https://linkedin.com/company/huggingface)
- **Discord**: Join our vibrant community on Discord for discussions and support.

---

Let’s build the future of AI, together.
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
### The AI Community Building the Future

At Hugging Face, we believe in the transformative power of artificial intelligence and machine learning. Our platform serves as a collaborative space for developers, researchers, and enthusiasts to create, share, and enhance machine learning models and applications. 

---

## Key Features

- **Extensive Model Repository**: Access over 1 million models across various modalities, including text, images, video, audio, and 3D.
- **Datasets**: Browse a vast collection of 250,000+ datasets to support your machine learning projects.
- **Spaces**: Create and deploy applications effortlessly with our Spaces functionality, featuring tools like DeepSite v2 and more.
- **Community Driven**: Join a thriving community of over 50,000 organizations, including leaders from Google, Microsoft, and Amazon.

---

## Company Culture

We are committed to fostering a culture of inclusivity, collaboration, and innovation. Our community-driven approach encourages contributions from all backgrounds, ensuring that our tools and models are continuously improving. Hugging Face is not just a platform; it's a community dedicated to democratizing access to machine learning.

- **Open Source Philosophy**: Our projects, including Transformers and Diffusers, are built and maintained with community input to create state-of-the-art ML tooling.
- **Supportive Environment**: We prioritize knowledge sharing and provide extensive documentation, forums, and resources to help users navigate their machine learning journey.

---

## Customers

Our diverse clientele ranges from small startups to large enterprises. We empower organizations to build innovative AI solutions with enterprise-grade security and support. Key customers include:

- AI at Meta
- Google
- Microsoft
- Amazon
- Grammarly

With over 50,000 organizations leveraging our platform, we are helping shape the future of machine learning.

---

## Careers at Hugging Face

As we continue to grow, we are always on the lookout for passionate individuals to join our team. We offer various roles that invite creativity and support personal and professional development.

- **Diverse Opportunities**: Roles in engineering, product management, community engagement, and more.
- **Collaborative Environment**: Work with cutting-edge technology alongside experts in AI and machine learning.
- **Competitive Benefits**: We offer comprehensive benefits packages, fostering work-life balance and professional growth.

If you are excited about AI and want to make a meaningful impact, explore our job openings and become part of the Hugging Face community!

---

## Get Involved!

Join us on our mission to democratize AI. Explore our resources, participate in community discussions, or contribute to our open-source projects. Together, we can build a brighter future through technology.

### Connect with Us
- [Website](https://huggingface.co)
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/hugging-face)

---

Join Hugging Face and be a part of the AI revolution!

In [22]:
# A more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
 and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
 Include details of company culture, customers and careers/jobs if you have the information."

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}


# Welcome to Hugging Face: The Huggable Future of AI!

🌟 **Where AI Goes for a Cuddle!** 🌟

Hey there, aspiring AI-ologist! If you've ever found yourself daydreaming about a community where humans and AI snuggle together to build something beautiful and pioneering, then stop right there! You’re in the right place. Let us introduce you to **Hugging Face** – your new favorite AI family!

## 🤖 **What We Do**

At Hugging Face, we believe that building the future can be a *team sport*! Our platform is the ultimate playground for the machine learning community. With collaboration on over **1 Million+ models** and **250k+ datasets**, we’re like the “Hugging Wonderland” for machine learning fanatics.

### Quick Highlights:
- **Trending Models:** 
  - *FLUX.1-Kontext-dev:* Updated 8 days ago, 155k fans (every hug counts 🥰).
  - *Hunyuan-A13B-Instruct:* Because sometimes, you need a smart bear to take direction (instructed with *love*, of course).
  
- **Snap, Share & Hug:** Whether it’s text, image, audio, video, or even 3D fun—we’ve got it all!

## 🌈 **Company Culture: A Hug Above the Rest!**

Here at Hugging Face, we significantly value **collaboration, creativity, and a generally chill vibe**. Our team is a colorful mosaic of talents and backgrounds – think of it as a techie talent show minus the awkwardness! We work together, learn from each other, and use humor to solve perplexing puzzles.

### ✈️ **Perks of Joining the Hug:**
- Super friendly teammates who provide their best bear hugs (metaphorical, of course).
- A space where everyone's ideas count, kind of like a brainstorming cuddle puddle!
- Opportunities to work on meaningful, impactful projects that would make even your AI teddy bear proud.

## 👫 **Join the Hugging Family!**

**Careers at Hugging Face** are not just about code and algorithms—you'll also get the chance to innovate, inspire, and incorporate your love of **AI goodness** into your work life! Current openings range from model wranglers to data explorers, and yes—AI whisperers!

🔍 **Have you got what it takes?** 
Check out our careers page: [Jobs at Hugging Face](#)

## 🚀 **Who Loves Us?**

Over **50,000 organizations** can’t get enough of our warm embraces— from **AI2** to **Google**, to **Microsoft**! They all recognize the power of collaboration and creativity in the world of machine learning.

## 🏆 **Why Choose Hugging Face?**

- Collaborate on **enterprise-grade solutions** with security snuggles.
- **Affordable pricing!** Cloud resources as low as **$0.60/hour** (because who doesn’t love a bargain?).
- **Community support** that really means “we’ve got your back!”

So, whether you're a customer, investor, or simply someone looking to make their career dreams come true in AI, let’s transform the technological landscape together! 

Join us and help build a future where every hug brings innovation, creativity, and a whole lot of fun!

---

### 🤗 **Hugging Face: Wherever You Are, We’re Here for the Hugs!** 

Feel free to browse through our services, look at our endless models, or just drop us a line to say ‘hey’! Remember, we’re just a hug away!

---

**Find us here:**
- [Website](https://huggingface.co)
- [Twitter](https://twitter.com/huggingface)
- [Discord](https://discord.com/invite/huggingface)

Hugs,  
**The Hugging Face Team** 

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>